In [ ]:
import os

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchinfo
import torchvision
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from torch.utils.data import DataLoader, random_split
from torchinfo import summary
from torchvision import datasets, transforms
from tqdm import tqdm

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using {device} device.")

We'll work with images of crop disease from Uganda which we prepared in the previous lesson. You may remember that we created an undersampled dataset that has a uniform distribution across classes. Let's use that dataset.

The data is in the data_p2 folder within which is the data_undersampled folder. In that folder we have the train folder that contains the training data.







**Task 2.3.1: Assign train_dir the path to the training data. Follow the pattern of data_dir.**

In [ ]:
data_dir = os.path.join("data_p2", "data_undersampled")
train_dir = os.path.join(data_dir,"train")

print("Data Directory:", data_dir)
print("Training Data Directory:", train_dir)

**Task 2.3.2: Create a list of class names using os.listdir.**



In [ ]:
classes = os.listdir(train_dir)

print("List of classes:", classes)

Following what we did in the previous lesson to standardize the images, we'll again use the same set of transformations:

Convert any grayscale images to RGB format with a custom class
Resize the image, so that they're all the same size (we chose
 x
)
Convert the image to a Tensor of pixel values
Normalize the data (we normalize each color channel separately)
Here's the custom transformation that we've used before which converts images to RGB format:

In [ ]:
class ConvertToRGB(object):
    def __call__(self, img):
        if img.mode != "RGB":
            img = img.convert("RGB")
        return img


**Task 2.3.3: Complete the transformation pipeline below. It's missing the last two steps (converting images to PyTorch tensors and normalizing them). In the normalization step, make sure to use the mean and std values from the previous lesson.**

In [ ]:
# Define transformation to apply to the images
transform_normalized = transforms.Compose(
    [
        ConvertToRGB(),
        transforms.Resize((224, 224)),
        # Convert images to tensors
        # ...
        # Normalize the tensors (copy the mean and std from previous lesson!)
        transforms.ToTensor() ,
        transforms.Normalize(
    mean=[0.4326, 0.4953, 0.3120],
    std=[0.2178, 0.2214, 0.2091]
)
    ]
)

print(type(transform_normalized))
print("--------------")
print(transform_normalized)

**Task 2.3.4: Make a normalized dataset using ImageFolder from datasets and print the length of the dataset.**

In [ ]:
dataset = datasets.ImageFolder(root=train_dir,transform=transform_normalized)

print('Length of dataset:', len(dataset))


**Train and validation splitting**
We'll follow good practice and divide our data into two parts. One part will be the data we'll train our model on. The second part will be used to evaluate the model on images it hasn't seen in training.

This is an important step in order for us to check how good the model is. If it makes good predictions on the training data but not on the validation data, we'll know the model's overfit.

**Task 2.3.5: Use random_split to create a 80/20 split (training dataset should have 80% of the data, validation dataset should have 20% of the data).**

In [ ]:
# Important, don't change this!
g = torch.Generator()
g.manual_seed(42)

train_dataset, val_dataset = random_split(dataset,[0.8,0.2], generator=g)

print("Length of training dataset:", len(train_dataset))
print("Length of validation dataset:", len(val_dataset))

**Task 2.3.6: Compute the length of the entire dataset, the training dataset and the validation dataset. We've added the code that computes the percentage of data that's training data and percentage that's validation.**

In [ ]:
length_dataset = len(dataset)
length_train = len(train_dataset)
length_val = len(val_dataset)

percent_train = np.round(100 * length_train / length_dataset, 2)
percent_val = np.round(100 * length_val / length_dataset, 2)

print(f"Train data is {percent_train}% of full data")
print(f"Validation data is {percent_val}% of full data")

**Task 2.3.7: Use class_counts function on the entire dataset and visualize the results with a bar chart. Note that computing dataset_counts may take a long time.**

In [ ]:
from training import class_counts

dataset_counts = class_counts(dataset)
dataset_counts.sort_values().plot(kind="bar")

# Make a bar chart from the function output

# Add axis labels and title
plt.xlabel("Class Label")
plt.ylabel("Frequency [count]")
plt.title("Distribution of Classes in Entire Dataset");








Task 2.3.8: Use the class_counts function and pandas plotting to make the same plot for the training data.

In [ ]:
train_counts = class_counts(train_dataset)
# Make a bar chart from the function output
train_counts.sort_values().plot(kind="bar")

# Add axis labels and title
plt.xlabel("Class Label")
plt.ylabel("Frequency [count]")
plt.title("Distribution of Classes in Training Dataset");

**Task 2.3.9: Use the class_counts function and pandas plotting to get the breakdown across classes for the validation split.**

In [ ]:
val_counts = class_counts(val_dataset)

# Make a bar chart from the function output
val_counts.sort_values().plot(kind="bar")
# Add axis labels and title
plt.xlabel("Class Label")
plt.ylabel("Frequency [count]")
plt.title("Distribution of Classes in Validation Dataset");

**Task 2.3.10: Create the training loader. Make sure to set shuffling to be on.**

In [ ]:
batch_size = 32

train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

print(type(train_loader))

**Task 2.3.11: Create the validation loader. Make sure to set shuffling to be off.**

In [ ]:
val_loader = DataLoader(val_dataset,batch_size=batch_size,shuffle=False)

print(type(val_loader))








Task 2.3.12: Print the shape of a batch of images and the shape of a batch of labels. **bold text**

In [ ]:
data_iter = iter(train_loader)
images, labels = next(data_iter)

image_shape = images.shape
print("Shape of batch of images", image_shape)

label_shape = labels.shape
print("Shape of batch of labels:", label_shape)

**Building a Convolutional Neural Network**
As we learned in the previous project, a network architecture suitable for image classification is the convolutional neural network (CNN). It primarily consists of a sequence of convolutional and max pooling layers. These layers are followed by some fully connected layers and an output layer.

Let's build a CNN!

Same as previously, we'll use the nn.Sequential class from PyTorch to define the architecture. We'll start with an empty model and append layers to it one by one.

In [ ]:
model = torch.nn.Sequential()

Task 2.3.13: Define the first convolutional layer of our network. Remember that we have three color channels, so set in_channels=3. Use
 kernels, each of siz
 and set padding to


In [ ]:
# Convolutional layer 1 (sees 3x224x224 image tensor)
conv1 = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,padding=1)
model.append(conv1)

print(model)

In [ ]:
max_pool1 = nn.MaxPool2d(2, 2)
model.append(torch.nn.ReLU())
model.append(max_pool1)

**Task 2.3.14: Define another convolutional layer of our network. This one should hav**e


In [ ]:
# Convolutional layer 2 (sees 16x112x112 tensor)
conv2 = nn.Conv2d(16,32,3,padding=1)
max_pool2 = nn.MaxPool2d(2, 2)
model.append(conv2)
model.append(torch.nn.ReLU())
model.append(max_pool2)

print(model)


Task 2.3.15: Define the last convolutional layer of our network. This one should have 64
 kernels 3. Again use kernels of size
 and padding of 1


In [ ]:
# Convolutional layer 3 (sees 32x56x56 tensor)
conv3 = nn.Conv2d(32,64,3,padding=1)
max_pool3 = nn.MaxPool2d(2, 2)
model.append(conv3)
model.append(torch.nn.ReLU())
model.append(max_pool3)

print(model)

In [ ]:
model.append(torch.nn.Flatten())
model.append(nn.Dropout(0.5))

**Task 2.3.16: Add a Linear layer to the model. You'll need to tell it the size of the input, and how many neurons we want in the layer (let's use
 500 neurons)**

In [ ]:
# Linear layer (64 * 28 * 2**8 -> 500)
linear1 = torch.nn.Linear(64*28*28,500)
model.append(linear1)
model.append(torch.nn.ReLU())
model.append(torch.nn.Dropout())

print(model)








**Task 2.3.17: Add the output layer to the model.**

In [ ]:
# Linear layer (500 -> 5)
output_layer = nn.Linear(500,5)
model.append(output_layer)

print(model)

**Task 2.3.18: Define cross-entropy as the loss function and set Adam optimizer to be the optimizer. You can use the default learning rate lr=0.001.**

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

print(loss_fn)
print("----------------------")
print(optimizer)

In [ ]:
model.to(device)

In [ ]:
height = 224
width = 224
summary(model, input_size=(batch_size, 3, height, width))

In [ ]:
from training import train

**Task 2.3.19: Use the train function to train the model for 15 epochs. Note that this may take a long time to run.**

In [ ]:
model = torch.load("model_trained.pth", weights_only=False)
model.to("cuda")

In [ ]:
df = pd.read_csv('post_train_evaluation_metrics.csv')

In [ ]:
train_losses = df['Train Loss']
valid_losses = df['Validation Loss']

train_accuracies = df['Train Accuracy']
valid_accuracies = df['Validation Accuracy']

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Training Loss")
plt.plot(valid_losses, label="Validation Loss")
plt.title("Loss over epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label="Training Accuracy")
plt.plot(valid_accuracies, label="Validation Accuracy")
plt.title("Accuracy over epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.show()

Oh no, we’re overfitting! 🤯

Overfitting occurs when a model learns the training data too well, capturing noise and details to the extent that it negatively impacts the model’s performance on new data. The symptoms of overfitting in our training results are:

High training accuracy: The model performs exceptionally well on the training data.
Increasing validation loss: Despite improvements in training loss and accuracy, the validation loss starts to increase after reaching a certain point.
Stagnant or decreasing validation accuracy: The model’s ability to generalize to unseen data does not improve or worsens as training progresses.
Addressing Overfitting

To mitigate overfitting, consider the following strategies:

Data Augmentation: Augment the training data by applying random transformations (e.g., rotations, scaling, flips) to generate new training samples. This can help the model generalize better.

Dropout: Introduce dropout layers into our network. Dropout randomly sets a fraction of input units to 0 during training, which helps prevent the model from becoming too reliant on any single feature.

Regularization: Apply regularization techniques, such as L1 or L2 regularization, which add a penalty on the magnitude of network parameters. This can discourage complex models that overfit.

Early Stopping: Monitor the model’s performance on a validation set and stop training when the validation loss starts to increase, which is a sign that the model's beginning to overfit.

Reduce Model Complexity: Simplify your model by reducing the number of layers or the number of units in the layers. A simpler model may generalize better.

Use More Data: If possible, adding more data can help the model learn better and generalize well to new, unseen data.

Batch Normalization: Although primarily used to help with training stability and convergence, batch normalization can sometimes also help with overfitting by regularizing the model somewhat.

Cross-validation: Helps prevent overfitting by testing the model on different parts of the data, ensuring it performs well on new data.

**Task 2.3.20: Use the predict function from training.py to compute probabilities that our model predicts on the validation data. The rest of the code provided will take these probabilities and compute the predicted classes.**

In [ ]:
from training import predict

probabilities_val = predict(model,val_loader,device)
predictions_val = torch.argmax(probabilities_val, dim=1)

print(predictions_val)

In [ ]:
targets_val = torch.cat(
    [labels for _, labels in tqdm(val_loader, desc="Get Labels")]
).to(device)

In [ ]:
cm = confusion_matrix(targets_val.cpu(), predictions_val.cpu())

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

# Set figure size
plt.figure(figsize=(10, 8))

disp.plot(cmap=plt.cm.Blues, xticks_rotation="vertical")
plt.show()